In [162]:
# Load modules
import os
import pandas as pd
import re
import ast
import glob
from pathlib import Path
import shutil

path = "C:/Users/19049/coding_files/data/CFM5.3_traces.ts"
subset = "C:/Users/19049/coding_files/data/subset.ts"
folder = "C:/Users/19049/coding_files/faults/"
wb_2b = "C:/Users/19049/coding_files/wb_2b/"

to_open = path

#currently the path is saevd as a .ts file, it'd be nice to have a conversion automatically from utm

In [163]:
#I did not write this https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder
#Just a tool to clear the faults folder- but could be useful later when reading in new data
def Delete_faults_content():
    [f.unlink() for f in Path("C:/Users/19049/coding_files/faults").glob("*") if f.is_file()]
Delete_faults_content()
def Delete_wb_2b_content():
    [f.unlink() for f in Path("C:/Users/19049/coding_files/wb_2b").glob("*") if f.is_file()]
Delete_wb_2b_content()

In [164]:
# @title
#what i've got:
#strip the crap from the fault name line (the tag and excess quotes, can expand to more stuff)
#open a fault folder to stick the coords in

#What i want:
#how tf do I put the coords into the new files now- I think I can use the current name and use the make_new_fault as a ticker, so the "put number in folder" function is dumb and every time it sees
#a coord it will just add it to the fault that is currently "name" and that should change since the for loop goes down the list in order
#it'd be nice to get a counter as a discrepancy check

#strips "> " and extra "" from line and sets it to global name
def redefine_name():
    global name
    t1_name = line
    t2_name = re.sub("[> ]", "", t1_name)
    name = ast.literal_eval(t2_name)

#makes an empty pandas dataframe using the empty list l
def make_new_fault():
    global l1
    l1 = pd.DataFrame(l,columns = ['Easting(m)','Northing(m)','Elevation(m)'])

#the line being read is delimited and turned from a line of strings into a line of floats
#then it is apended to the l1 dataframe
def number_line():
    string_line = line.split()
    float_list = list(map(float, string_line))
    l1.loc[len(l1)] = float_list

#takes the finished l1 dataframe and names it the current fault trace, making it a csv    
def save_line():
    l1.to_csv(folder+name+".csv")
    
#what we want to do is move the current fault in names, to a folder by providing the current path and the path to the folder
def move():
    shutil.move(folder+fault, wb_2b+fault)

In [165]:
#l being here resets the dataframe every time it's run, though I can probably just define make_new_fault pd.dataframe([],columns....) instead of making l=[]
l = []
with open(to_open, encoding='utf-8') as f:
    for line in f:
        if line.startswith("> "):
            is_number = False
            redefine_name()
            make_new_fault()
        elif line.startswith("#"):
            pass
        else:
            is_number = True
            while is_number == True:
                if not line.startswith("> "):
                    number_line()
                break
            save_line()

In [166]:
#13 faults that are included in the project that we want to make worlbuilder files of
# format: '*_fault',\n
want = [
    'Panamint_Valley_fault',
    'Southern_Death_Valley_fault',
    'Black_Mountain_fault',
    'Owl_Lake_fault',
    'Hunter_Mountain_fault',
    'Ash_Hill_fault',
    'Independence_Sierra_Nevada_fault', #Gotta add the _fault identification bit cause of
    'Independence_fault',               #these two lines
    'Little_Lake_fault',
    'Airport_Lake_fault',
    'Eastern_Little_Lake_north_north_east_cross_fault',
    'Owens_Valley_fault',
    'Northern_Death_Valley_fault'
    ]

In [161]:
#moves the files in the faults folder with the short names defined in want to the worldbuilder to be folder (wb_2b)
#now when I say move, it deletes the file in the fault folder, so it's not there anymore and has to be remade
fault_names = os.listdir(r"C:\Users\19049\coding_files\faults")
for fault in fault_names:
    for short in want:
        if short in fault:
            move()
        else:
            pass

In [ ]:
#I"m thinking that now we can identify the faults we want out of the pile, we put them into another folder, which we can then run indescriminate code on, which I suppose the purpose of that code
#would be to convert the trace data to a worldbuilder file format? which I think daniel has.